In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
%matplotlib qt5

In [2]:
import sys

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *

In [5]:
fmin = 5.725e9
fMAX = 5.875e9
f0 = 5.8e9
B = fMAX - fmin
T = 600e-6
alpha = 2 * (fMAX - fmin) / T
c = 3e8

In [6]:
t = np.arange( 0, T, 100e-9)

In [7]:
t2 = np.arange( -T/2, T/2, 100e-9)

In [28]:
t2.shape

(6000,)

In [8]:
rampsPerFile = 1500
samplesPerRamp = 6000
samplesPerUpRamp = 3000
nbFiles = 10
adc = np.zeros( nbFiles * rampsPerFile * samplesPerRamp, dtype=complex )

In [9]:
def sr( t, r ):
    
    tau = 2 * r / c
    
    y = np.zeros( t.shape, dtype=complex )
    
    # up ramp
    y[0:samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * alpha * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * alpha * t[0:samplesPerUpRamp] * tau )
    
    # down ramp
    y[samplesPerUpRamp:2*samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * (-alpha) * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * (-alpha) * t[samplesPerUpRamp:2*samplesPerUpRamp] * tau )
    
    return y

In [10]:
def srn( t, r, n ):
    
    tau = 2 * r / c
    
    y = np.zeros( t.shape, dtype=complex )
    
    # up ramp
    y[0:samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * alpha * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * alpha * (t[0:samplesPerUpRamp] + n*T) * tau )
    
    # down ramp
    y[samplesPerUpRamp:2*samplesPerUpRamp] = np.exp( -1j * 2  * np.pi * f0 * tau ) \
    * np.exp( 1j * np.pi * (-alpha) * tau**2 ) \
    * np.exp(  -1j * 2 * np.pi * (-alpha) * (t[samplesPerUpRamp:2*samplesPerUpRamp] + n*T) * tau )
    
    return y

Let us build the equivalent of 10 files of data

In [11]:
a = 0.603354564402
b = a * 180 / (30 * np.pi) # see the Newton method at the bottom of the notebook
def wa( az, rg0 ):
    Rn = ( rg0**2 + az**2 )**0.5
    wa = np.sinc( b * np.arccos( rg0 / Rn ) )**2
    return wa

In [20]:
V = 40
hPlane = 90
hCorner = 90
gr_corner = 100
rg_corner = ( (hPlane - hCorner)**2 + gr_corner**2 )**0.5
az_corner = 150
noise = 0
print("rg_corner = {:.2f}, gr_corner = {:.2f}".format(rg_corner, gr_corner))

rg_corner = 100.00, gr_corner = 100.00


In [21]:
Laz = nbFiles * rampsPerFile * T * V
phi = np.arctan( (Laz/2) / rg_corner )
print( "Laz = {:.0f}, phi = {:.1f}°".format( Laz, phi * 180 / np.pi ) )

Laz = 360, phi = 60.9°


In [33]:
for n in range( nbFiles * rampsPerFile):
    azn = n * T * V
    Rn = ( rg_corner**2 + (azn - az_corner)**2 )**0.5
    adc[ n * samplesPerRamp : (n+1) * samplesPerRamp ] = srn( t, Rn, n ) * wa( azn - az_corner, rg_corner ) \
    + np.random.randn( 6000 ) * noise
    if n%1000 == 0:
        phi = np.arccos( rg_corner / Rn ) * 180 / np.pi
        print( "{}, wa = {:.1f}, phi = {:.1f}".format( n, wa( azn - az_corner, rg_corner ), phi ) )

0, wa = 0.0, phi = 56.3
1000, wa = 0.0, phi = 51.6
2000, wa = 0.0, phi = 45.6
3000, wa = 0.1, phi = 38.0
4000, wa = 0.3, phi = 28.4
5000, wa = 0.7, phi = 16.7
6000, wa = 1.0, phi = 3.4
7000, wa = 0.9, phi = 10.2
8000, wa = 0.5, phi = 22.8
9000, wa = 0.2, phi = 33.4
10000, wa = 0.0, phi = 42.0
11000, wa = 0.0, phi = 48.7
12000, wa = 0.0, phi = 54.1
13000, wa = 0.0, phi = 58.3
14000, wa = 0.0, phi = 61.7


In [34]:
adc_reshaped = adc.reshape( (nbFiles * rampsPerFile, samplesPerRamp) )

In [16]:
RD = build_rd_from_data_sim( np.real( adc_reshaped ), samplesPerRamp )
print( "RD.shape = {}".format( RD.shape ) )

shape of the samples matrix = (15000, 6000)
Data are NOT shifted
RD.shape = (15000, 1500)


# Save simulated data

In [17]:
firstFile = 0
lastFile = nbFiles - 1
data_dir = "/home/pleroy/DATA/SIMU"

In [35]:
np.save(data_dir + '/adc_files_{}_{}'.format(firstFile, lastFile), adc_reshaped[:, 0:3000])

In [19]:
np.save(data_dir + '/RD_files_{}_{}'.format(firstFile, lastFile), RD)

In [ ]:
coupling_RD = np.average(RD, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}".format(firstFile, lastFile), coupling_RD )

In [ ]:
samplesPerRamp

In [ ]:
ifft_RD = np.fft.ifft( RD, axis=1 )

In [ ]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( ifft_RD[::10,:] ) ) )
plt.colorbar()

In [ ]:
ifft_adc = np.fft.ifft( adc_reshaped, axis=1 )
print( "adc_reshaped.shape = {}, ifft_adc.shape = {}".format( adc_reshaped.shape, ifft_adc.shape ) )

In [ ]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( ifft_adc[::10,:] ) ) )
plt.colorbar()

In [ ]:
az_corner / V / T

In [ ]:
plt.figure()
plt.plot( 20 * np.log10( np.abs( ifft_RD[7750,:] ) ) )
plt.plot( 20 * np.log10( np.abs( ifft_RD[25000,:] ) ) )

In [ ]:
plt.figure()
plt.plot( 20 * np.log10( np.abs( ifft_RD[::10,500] ) ))
plt.plot( 20 * np.log10( np.abs( ifft_RD[::10,501] ) ))

In [ ]:
RD[100, 0:3000].size

In [ ]:
RD.shape

### Newton's method
f(x) = sin(x) / x

f'(x) = cos(x) / x - sin(x) / x^2

x(n+1) = x(n) - f(x(n)) / f'(x(n))

x(n+1) = x(n) - [f(x(n)) - y0] / f'(x(n))

In [ ]:
y0 = 0.5
def newton( xn ):
    f_xn = np.sinc( xn )
    #f_prime_xn = np.cos(xn) / xn - np.sin(xn) / xn**2
    f_prime_xn = np.cos(np.pi * xn) / xn - np.sin(np.pi*xn) / (np.pi*xn**2)
    xn_plus_1 = xn - ( f_xn - y0 ) / f_prime_xn
    return xn_plus_1

In [ ]:
x = 1
for k in range( 10 ):
    x = newton( x )
    print( x )

In [ ]:
a = 0.603354564402
b = a * 180 / (30 * np.pi)

In [ ]:
x = 30 * np.pi / 180
np.sinc( b * x )

### Analytic signal

In [ ]:
teta = np.arange( -10*np.pi, 10*np.pi, np.pi/10 )

In [ ]:
def as3( y ):
    fft_y = np.fft.fftshift( np.fft.fft( y ) )
    nbPoints = y.shape[0]
    nbPoints2 = int( nbPoints / 2 )
    fft_as_y = np.zeros( (nbPoints,), dtype=complex )
    fft_as_y[0] = fft_y[0]
    fft_as_y[ 1 : nbPoints2 ] = 2 * fft_y[ 1 : nbPoints2 ]
    fft_as_y[0] = fft_y[0]
    fft_as_y[nbPoints2] = fft_y[nbPoints2]
    return np.fft.ifft( np.fft.ifftshift( fft_as_y ) ) / 2

In [ ]:
def as2( y ):
    return signal.hilbert( y )/2

In [ ]:
def RD_realtocomp2( RD ):

    Nf  = RD.size

    # if Nf is a multiple of 2, fftshift(fftshift(X)) = X
    if ( ( Nf % 2 ) != 0 ):
        error( "Nf should be a multiple of 2" )

    h = np.zeros( Nf )
    h[ 0 ] = 1
    h[ int( Nf / 2) ] = 1
    h[ 1 : int(Nf / 2) ] = 2

    #RDs = np.fft.ifft( np.fft.fftshift( RD ) )
    RDs = np.fft.ifft( RD )

    RDh = RDs * h

    #RDc = np.fft.fftshift( np.fft.fft( RDh ) )
    RDc = np.fft.fft( RDh )

    return RDc/2

In [ ]:
phi0 = 30 * 180 / np.pi
y = np.cos(teta - phi0) + np.cos( 2 * teta - phi0)
y = teta
expy = np.exp(1j * (teta-phi0) ) / 2
as_y = RD_realtocomp2( y )
as2_y = as2( y )
as3_y = as3( y )
fft_y = np.fft.fft( y )
fft_expy = np.fft.fft( expy )

plt.figure()

plt.subplot(121)
plt.plot( np.real( np.fft.fft( as3_y ) ), "D-", label="as3_y" )
plt.plot( np.real( np.fft.fft( as2_y ) ), '.-', label="as2_y (hilbert)" )
plt.plot( np.real( np.fft.fft( as_y ) ), "x-", label="as_y (RD_realtocomp)", color="yellow" )
plt.plot( np.real( fft_y ), label="fft_y" )
plt.grid()
plt.legend()

plt.subplot(122)
plt.plot( np.imag( np.fft.fft( as3_y ) ), "D-" )
plt.plot( np.imag( np.fft.fft( as2_y ) ), '.-' )
plt.plot( np.imag( np.fft.fft( as_y ) ), "x-", color="yellow" )
plt.plot( np.imag( fft_y ) )
plt.grid()

In [ ]:
plt.figure()

as_y = RD_realtocomp( y )
as_y_ifftshift = np.fft.ifftshift(RD_realtocomp( y ))

plt.subplot(121)
plt.plot( np.real( np.fft.fft( as_y ) ), "x-", label="real as_y" )
plt.plot( np.real( np.fft.fft( as_y_ifftshift ) ), ".-", label="real as_y_ifftshift" )
plt.grid()
plt.legend()

plt.subplot(122)
plt.plot( np.imag( np.fft.fft( as_y ) ), "x-", label="imag as_y" )
plt.plot( np.imag( np.fft.fft( as_y_ifftshift ) ), ".-", label="imag as_y_ifftshift" )
plt.grid()

In [ ]:
plt.figure()

as_y = RD_realtocomp( y )
as_y_ifftshift = np.fft.ifftshift(RD_realtocomp( y ))

plt.subplot(121)
plt.plot( np.real( as_y ), "x-", label="real as_y" )
plt.plot( np.real( as_y_ifftshift ), ".-", label="real as_y_ifftshift" )
plt.grid()
plt.legend()

plt.subplot(122)
plt.plot( np.imag( as_y ), "x-", label="imag as_y" )
plt.plot( np.imag( as_y_ifftshift ), ".-", label="imag as_y_ifftshift" )
plt.grid()